In [232]:
import pandas as pd
import math
from collections import defaultdict,Counter
import matplotlib.pyplot as plt

In [233]:
df=pd.read_csv("./MetricFiles/idif.csv")

In [234]:
def realTraffic(df,minutes):
    traffic=defaultdict(list)
    trafficMax=dict()
    sourceAirports=list(df["departure_airport_icao_code"])
    destinationAirports=list(df["arrival_airport_icao_code"])
    rangeSize=max(list(df["idif_actual_runway_arrival"]))
    numberOfRanges=math.ceil(rangeSize/minutes)
    departureTimes=list(df["idif_actual_runway_departure"])
    arrivalTimes=list(df["idif_actual_runway_arrival"])
    for i in range(len(departureTimes)):
        depart=int(departureTimes[i]/minutes)
        if len(traffic[sourceAirports[i]]) == 0:
            traffic[sourceAirports[i]]=[0 for _ in range(numberOfRanges + 1)]
        traffic[sourceAirports[i]][depart]+=1
        arrive=int(arrivalTimes[i]/minutes)
        if len(traffic[destinationAirports[i]]) == 0:
            traffic[destinationAirports[i]]=[0 for _ in range(numberOfRanges + 1)]
        traffic[destinationAirports[i]][arrive]+=1
    for airport,trafficRanges in traffic.items():
        trafficMax[airport]=max(trafficRanges)
    return trafficMax

In [235]:
dfScheduled=pd.read_csv("./MetricFiles/idif_scheduled.csv")
with open("./MetricFiles/AerialTimeGD.txt") as f:
    data=f.readlines()[1:]
    data=list(map(lambda x:x[:-1],data))
    data=list(map(lambda x:[int(x.split(",")[0]),int(x.split(",")[1])],data))

In [236]:
def GATraffic(df,minutes):
    traffic=defaultdict(list)
    trafficMax=dict()
    scheduledDepartures=list(df["idif_scheduled_runway_departure"])
    sourceAirports=list(df["departure_airport_icao_code"])
    destinationAirports=list(df["arrival_airport_icao_code"])
    actualDeparture=[]
    actualArrival=[]
    for i in range(len(scheduledDepartures)):
        actualDeparture.append(scheduledDepartures[i] + data[i][1])
        actualArrival.append(actualDeparture[-1] + data[i][0])
    rangeSize=max(actualArrival)
    numberOfRanges=math.ceil(rangeSize/minutes)
    for i in range(len(scheduledDepartures)):
        depart=int(actualDeparture[i]/minutes)
        if len(traffic[sourceAirports[i]]) == 0:
            traffic[sourceAirports[i]]=[0 for _ in range(numberOfRanges + 1)]
        traffic[sourceAirports[i]][depart]+=1
        arrive=int(actualArrival[i]/minutes)
        if len(traffic[destinationAirports[i]]) == 0:
            traffic[destinationAirports[i]]=[0 for _ in range(numberOfRanges + 1)]
        traffic[destinationAirports[i]][arrive]+=1
    for airport,trafficRanges in traffic.items():
        trafficMax[airport]=max(trafficRanges)
    return trafficMax

In [237]:
minutes=5
x=realTraffic(df,minutes)
y=GATraffic(dfScheduled,minutes)
c=0
for airport in x:
    if y[airport] <= x[airport]:
        c+=1
print(f"Traffic in GA is better than or equal to real data in {c*100/len(x)} % of the airports")

Traffic in GA is better than or equal to real data in 82.4665676077266 % of the airports
